`ApJdataFrames` Luhman2010
---
`Title`: The Disk Population of the Taurus Star-Forming Region
`Authors`: Kevin L Luhman, et al.

Data is from this erratum to this paper:
http://iopscience.iop.org/0067-0049/189/2/353/article#apjs363478t1

In [1]:
%pylab inline
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

## Table 13- *corrected* (see erratum)

In [4]:
names = ['Spectral Type','J - H','H - K_s','K_s - [3.6]','[3.6] - [4.5]',
         '[4.5] - [5.8]','[5.8] - [8.0]','[8.0] - [24]']
tbl13 = pd.read_csv("http://iopscience.iop.org/0067-0049/189/2/353/suppdata/apjs363478t1_ascii.txt",
                   sep='\t', skiprows=range(4), names = names)
tbl13.drop(0, axis=0, inplace=True)
tbl13.head()

,Spectral Type,J - H,H - K_s,K_s - [3.6],[3.6] - [4.5],[4.5] - [5.8],[5.8] - [8.0],[8.0] - [24]
1,K4,0.53,0.10,0.06,0,0.04,0,0.04
2,K5,0.61,0.12,0.08,0,0.04,0,0.06
3,K6,0.66,0.14,0.10,0,0.04,0,0.08
4,K7,0.68,0.15,0.10,0,0.04,0,0.11
5,M0,0.70,0.16,0.10,0,0.04,0,0.13


In [5]:
from gully_custom import specType

In [6]:
tbl13['SpT'] = specType(tbl13['Spectral Type'])

##Save the data

    !!mkdir ../data/Luhman2010

In [8]:
tbl13.to_csv('../data/Luhman2010/tbl13_fixed_erratrum.csv', index=False)

The end.